In [1]:
import pandas as pd
import numpy as np
import sys
import random
from sqlalchemy import create_engine

from snorkel.labeling import labeling_function

## Load Data

### DataFrame

In [2]:
## load the data by choosing a random 10% of the data
## is the data ordered? am I missing a large chunk due to my partial loading?
# p = 0.01
# responses = pd.read_csv('../data/moralmachine/SharedResponses.csv', skiprows=lambda i: i>0 and random.random() > p)

### SQL Engine

In [3]:
# access SQL DB with data
engine = create_engine("sqlite:///../data/moralmachine.db", echo=False)

In [4]:
# demo
# responses = pd.read_sql("SELECT * FROM sharedresponses ORDER BY RANDOM() LIMIT 100000", con=engine)
# responses.columns

In [5]:
# ## For viewing the possible values for each feature
# for feature in responses.columns:
#     print(feature)
#     print(responses[feature].unique())
#     print(responses[feature].unique().shape)

In [6]:
# ## What does a single user's session/response look like?
# responses_random = responses[responses['ScenarioTypeStrict'] == 'Random']
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     ## see a random user's session
# #     display(responses_random[responses_random['ExtendedSessionID'] == responses_random['ExtendedSessionID'].sample().values[0]])
    
#     ## see the user with nth most responses
#     n = 100
#     display(responses_random[responses_random['ResponseID'] == responses_random.groupby(by='ResponseID').size().sort_values(ascending=False).index[n]]) 

In [7]:
# ## group alternatives pairwise!
# # take only responses where both alternatives are present in the random sample - for a full sample, this step isn't necessary
# responses_grouped = responses.groupby(by='ResponseID').filter(lambda g: g.shape[0] < 2)
# responses_grouped.shape

## Data Exploration

In [8]:
# responses.describe()

## Snorkel Labeling
Using https://www.snorkel.org/use-cases/01-spam-tutorial

In [9]:
# query random sample of responses grouped by ResponseID; only take responses for which both instances are present
query = """
    --SELECT * FROM (
        SELECT * FROM sharedresponses
            WHERE ScenarioTypeStrict LIKE 'Random'
        ORDER BY RANDOM()
        LIMIT 1000000
    --)
    --GROUP BY ResponseID, ExtendedSessionID, UserID
    --HAVING COUNT(ResponseID) > 1
"""
df = pd.read_sql(query, con=engine).groupby(by='ResponseID').filter(lambda g: g.shape[0] > 1).sort_values('ResponseID')
df

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
28075,25ufd785biRZFDSeG,-1006643273_146292868172649.0,146292868172649,3,1,1,0,1,Rand,Random,...,1,0,0,0,0,0,0,0,1,0
44754,25ufd785biRZFDSeG,-1006643273_146292868172649.0,146292868172649,3,0,1,0,2,Rand,Random,...,1,0,0,0,0,0,0,0,0,0
96673,27pTCNZcXdvNXy3NF,1900423641_930198056008106.0,930198056008106,4,1,0,0,2,Rand,Random,...,0,1,0,0,0,0,0,0,0,0
87895,27pTCNZcXdvNXy3NF,1900423641_930198056008106.0,930198056008106,4,0,0,1,0,Rand,Random,...,1,0,0,0,0,0,0,0,0,1
70700,2CYcBT8meYg4kCubi,1271980589_3108928246.0,3108928246,13,0,0,1,0,Rand,Random,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5256,zq7duZsMx8QGr6siY,-1543001569_3335841152.0,3335841152,11,0,0,1,0,Rand,Random,...,0,0,0,0,0,0,0,0,0,0
29642,zvpdJ4NZpEijrNt8C,-1434791710_3467380571.0,3467380571,8,1,0,1,0,Rand,Random,...,1,0,1,0,0,0,0,0,1,0
82829,zvpdJ4NZpEijrNt8C,-1434791710_3467380571.0,3467380571,8,0,0,0,2,Rand,Random,...,0,0,1,0,0,0,0,0,0,0
75897,zx6LE356enPj38SKE,-2010755996_3069458898.0,3069458898,7,1,0,0,2,Rand,Random,...,0,0,0,1,0,1,0,0,0,0


Need to have both alternatives in the same tuple, marking them by whether or not they are the intervention - specifically the factors:
> 'NumberOfCharacters', 'DiffNumberOfCharacters', 'Saved', 'Template', 'DescriptionShown',
'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
'Cat'

In [10]:
df[["ResponseID","Intervention","Saved"]]

,ResponseID,Intervention,Saved
28075,25ufd785biRZFDSeG,1,0
44754,25ufd785biRZFDSeG,0,1
96673,27pTCNZcXdvNXy3NF,1,0
87895,27pTCNZcXdvNXy3NF,0,1
70700,2CYcBT8meYg4kCubi,0,0
...,...,...,...
5256,zq7duZsMx8QGr6siY,0,1
29642,zvpdJ4NZpEijrNt8C,1,0
82829,zvpdJ4NZpEijrNt8C,0,1
75897,zx6LE356enPj38SKE,1,1


In [11]:
scenario_fields = [
    'ResponseID', 'Barrier', 'NumberOfCharacters', 'DiffNumberOFCharacters', 'Saved', 'DescriptionShown', 'LeftHand', 'Man', 'Woman', 'Pregnant', 'Stroller', \
    'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', \
    'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat'
]
intervention = df[df['Intervention'] == 1][scenario_fields].set_index('ResponseID')
no_intervention = df[df['Intervention'] == 0][scenario_fields].set_index('ResponseID')
df_joined = intervention.join(no_intervention, lsuffix='_int', rsuffix='_noint', how='inner')
df_joined = df[[col for col in df.columns if col not in scenario_fields or col == 'ResponseID']].set_index('ResponseID').join(df_joined)
df_joined['Intervened'] = (df_joined['Saved_int'] == 1).astype(int)
df_joined = df_joined.drop(axis='columns', labels=['Saved_{}'.format(s) for s in ['int', 'noint']])
df_joined

,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,CrossingSignal,AttributeLevel,ScenarioTypeStrict,ScenarioType,DefaultChoice,...,Criminal_noint,MaleExecutive_noint,FemaleExecutive_noint,FemaleAthlete_noint,MaleAthlete_noint,FemaleDoctor_noint,MaleDoctor_noint,Dog_noint,Cat_noint,Intervened
ResponseID,,,,,,,,,,,,,,,,,,,,,
25ufd785biRZFDSeG,-1006643273_146292868172649.0,146292868172649,3,1,1,1,Rand,Random,Random,,...,0,0,0,0,0,0,0,0,0,0
25ufd785biRZFDSeG,-1006643273_146292868172649.0,146292868172649,3,0,1,2,Rand,Random,Random,,...,0,0,0,0,0,0,0,0,0,0
27pTCNZcXdvNXy3NF,1900423641_930198056008106.0,930198056008106,4,1,0,2,Rand,Random,Random,,...,0,0,0,0,0,0,0,0,1,0
27pTCNZcXdvNXy3NF,1900423641_930198056008106.0,930198056008106,4,0,0,0,Rand,Random,Random,,...,0,0,0,0,0,0,0,0,1,0
2CYcBT8meYg4kCubi,1271980589_3108928246.0,3108928246,13,0,0,0,Rand,Random,Random,,...,0,0,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zq7duZsMx8QGr6siY,-1543001569_3335841152.0,3335841152,11,0,0,0,Rand,Random,Random,,...,0,0,0,0,0,0,0,0,0,0
zvpdJ4NZpEijrNt8C,-1434791710_3467380571.0,3467380571,8,1,0,0,Rand,Random,Random,,...,0,1,0,0,0,0,0,0,0,0
zvpdJ4NZpEijrNt8C,-1434791710_3467380571.0,3467380571,8,0,0,2,Rand,Random,Random,,...,0,1,0,0,0,0,0,0,0,0


A standard train test split for testing:

In [12]:
from sklearn.model_selection import train_test_split

X = df_joined.drop(labels=["Intervened"], axis='columns', inplace=False)
y = df_joined["Intervened"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
X_test, X_dev, y_test, y_dev = train_test_split(X_test, y_test, test_size=0.2, random_state=1)

Some sample labeling functions:

In [13]:
sys.path.insert(1, '../heuristics')
import labeling_functions
import utils
import importlib
importlib.reload(labeling_functions)
importlib.reload(utils)

from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis


lfs = [
    labeling_functions.doctors,
    labeling_functions.utilitarian,
    labeling_functions.utilitarian_anthro,
    labeling_functions.inaction,
    labeling_functions.pedestrians,
    labeling_functions.females,
    labeling_functions.fitness,
    labeling_functions.status,
    labeling_functions.legal,
    labeling_functions.illegal,
    labeling_functions.youth,
    labeling_functions.criminals,
    labeling_functions.homeless,
    labeling_functions.pets,
    labeling_functions.spare_strollers,
    labeling_functions.spare_girl,
    labeling_functions.spare_boy,
    labeling_functions.spare_pregnant
]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=X_train)
L_dev = applier.apply(df=X_dev)
L_valid = applier.apply(df=X_val)
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=y_dev.values).sort_values("Correct", ascending=False)

/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 275/275 [00:00<00:00, 891.44it/s]


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
utilitarian_anthro,2,"[0, 1]",0.782609,0.782609,0.666667,34,20,0.629630
utilitarian,1,"[0, 1]",0.811594,0.811594,0.710145,33,23,0.589286
pedestrians,4,"[0, 1]",0.855072,0.855072,0.782609,32,27,0.542373
inaction,3,[0],1.000000,1.000000,0.869565,26,43,0.376812
fitness,6,"[0, 1]",0.565217,0.565217,0.536232,26,13,0.666667
youth,10,"[0, 1]",0.608696,0.608696,0.550725,26,16,0.619048
females,5,"[0, 1]",0.579710,0.579710,0.521739,25,15,0.625000
spare_strollers,14,"[0, 1]",0.260870,0.260870,0.246377,13,5,0.722222
status,7,"[0, 1]",0.405797,0.405797,0.318841,13,15,0.464286
doctors,0,"[0, 1]",0.347826,0.347826,0.333333,12,12,0.500000


## Aggregation

Recall that there are no true labels for this problem - really, we're just measuring similarity of the heuristic labels to real voter's responses. 

**Baseline**: majority label voting.

In [14]:
from snorkel.labeling import MajorityLabelVoter

model_majority = MajorityLabelVoter()
preds_train = model_majority.predict(L=L_train)
preds_train

array([ 1,  0, -1, ...,  1,  0, -1])

Snorkel label model.

In [15]:
from snorkel.labeling import LabelModel

# cardinality is num classes
model_label = LabelModel(cardinality=2, verbose=True)
model_label.fit(L_train=L_train, n_epochs=500, lr=.001, log_freq=100, seed=1)

In [16]:
for model in [model_majority, model_label]:
    acc = model.score(L=L_valid, Y=y_val.values, tie_break_policy="random")["accuracy"]
    print(f"{str(model)} {'Vote Accuracy:':<25} {acc*100:.1f}")

MajorityLabelVoter() Vote Accuracy:            61.5
LabelModel() Vote Accuracy:            68.0


In [17]:
from snorkel.analysis import get_label_buckets

probs_dev = model_label.predict_proba(L=L_dev)
preds_dev = probs_dev >= 0.5
buckets = get_label_buckets(y_dev.values, preds_dev[:, 1])

df_fn_dev = X_dev.iloc[buckets[(1, 0)]]
df_fn_dev["probability"] = probs_dev[buckets[(1, 0)], 1]

df_fn_dev.sample(5, random_state=3)

/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,CrossingSignal,AttributeLevel,ScenarioTypeStrict,ScenarioType,DefaultChoice,...,Criminal_noint,MaleExecutive_noint,FemaleExecutive_noint,FemaleAthlete_noint,MaleAthlete_noint,FemaleDoctor_noint,MaleDoctor_noint,Dog_noint,Cat_noint,probability
ResponseID,,,,,,,,,,,,,,,,,,,,,
4iLsmt7baxLoD4h8a,215064411_3336850889.0,3336850889,12,1,0,1,Rand,Random,Random,,...,0,0,0,0,0,1,0,0,0,1.413486e-11
BFzxsd5RPrvMrZMA4,-509851889_3629757071.0,3629757071,11,1,1,1,Rand,Random,Random,,...,0,0,0,1,0,0,0,1,0,5.731939e-07
3THcBzajbsZt2fvr9,-1415443789_5540630902943597.0,5540630902943600,8,0,0,1,Rand,Random,Random,,...,0,0,0,0,0,0,0,0,0,2.339071e-02
49sg2tPLQ7sfWBw6f,-350939289_3829534088189021.0,3829534088189020,9,0,0,0,Rand,Random,Random,,...,0,0,0,0,0,2,0,0,0,1.924312e-10
GH2eSL6nXu2q7JquD,-582555278_6020094797756707.0,6020094797756710,11,0,0,2,Rand,Random,Random,,...,0,0,0,0,0,0,0,1,0,3.561248e-07


Filter out unlabeled data points.

## Classification

https://www.snorkel.org/use-cases/01-spam-tutorial#5-training-a-classifier

### Featurization

In [18]:
# include user countries and other metadata or not? decided not - experts writing LFs are trying to decide most moral response, not predict what an individual from a certain background would do
# but if it helps generalization, technically useful... try both?
features = [
    "PedPed", "CrossingSignal", 'NumberOfCharacters_int', 'Man_int', 'Woman_int',
    'Pregnant_int', 'Stroller_int', 'OldMan_int', 'OldWoman_int', 'Boy_int',
    'Girl_int', 'Homeless_int', 'LargeWoman_int', 'LargeMan_int',
    'Criminal_int', 'MaleExecutive_int', 'FemaleExecutive_int',
    'FemaleAthlete_int', 'MaleAthlete_int', 'FemaleDoctor_int', 'Barrier_int',
    'MaleDoctor_int', 'Dog_int', 'Cat_int', 'Barrier_noint', 'NumberOfCharacters_noint',
    'Man_noint', 'Woman_noint', 'Pregnant_noint', 'Stroller_noint', 'OldMan_noint', 'OldWoman_noint',
    'Boy_noint', 'Girl_noint', 'Homeless_noint', 'LargeWoman_noint',
    'LargeMan_noint', 'Criminal_noint', 'MaleExecutive_noint',
    'FemaleExecutive_noint', 'FemaleAthlete_noint', 'MaleAthlete_noint',
    'FemaleDoctor_noint', 'MaleDoctor_noint', 'Dog_noint', 'Cat_noint', 
#     "Template", "UserCountry3", 'DescriptionShown_int', 'LeftHand_int', 'DescriptionShown_noint', 'LeftHand_noint'
]
cat_features = [
#     "Template", "UserCountry3"
]
num_features = [
    "PedPed", "CrossingSignal", "NumberOfCharacters_int", "NumberOfCharacters_noint", "Man_int", "Man_noint", "Woman_int", "Woman_noint", 
    "Pregnant_int", "Pregnant_noint", "Stroller_int", "Stroller_noint", "OldMan_int", "OldMan_noint", "OldWoman_int", "OldWoman_noint", "Boy_int", "Boy_noint",
    "LargeMan_noint", "LargeMan_int", "Criminal_int", "Criminal_noint", "MaleExecutive_int", "MaleExecutive_noint", "FemaleExecutive_int",
    "FemaleExecutive_noint", "Girl_int", "Girl_noint", "LargeWoman_int", "LargeWoman_noint", "FemaleAthlete_int", "FemaleAthlete_noint",
    "MaleAthlete_int", "MaleAthlete_noint", "FemaleDoctor_int", "FemaleDoctor_noint", "MaleDoctor_int", "MaleDoctor_noint", "Dog_int",
    "Dog_noint", "Cat_int", "Cat_noint", "Homeless_int", "Barrier_noint", "Homeless_noint", "Barrier_int"
#   'DescriptionShown_int', 'LeftHand_int', 'DescriptionShown_noint', 'LeftHand_noint'
]
# are there any nan?
X_train[num_features].describe()

,PedPed,CrossingSignal,NumberOfCharacters_int,NumberOfCharacters_noint,Man_int,Man_noint,Woman_int,Woman_noint,Pregnant_int,Pregnant_noint,...,MaleDoctor_int,MaleDoctor_noint,Dog_int,Dog_noint,Cat_int,Cat_noint,Homeless_int,Barrier_noint,Homeless_noint,Barrier_int
count,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000,...,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000,1097.000000
mean,0.104831,0.760255,3.015497,3.133090,0.136737,0.163172,0.158614,0.163172,0.161349,0.154057,...,0.135825,0.158614,0.159526,0.140383,0.164084,0.157703,0.125798,0.412944,0.147675,0.482224
std,0.306476,0.848973,1.468879,1.410521,0.376656,0.402764,0.391983,0.402764,0.394350,0.390307,...,0.350654,0.391983,0.392776,0.360430,0.427675,0.402680,0.358219,0.492588,0.384552,0.499912
min,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2.000000,4.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1.000000,2.000000,5.000000,5.000000,3.000000,2.000000,3.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000,3.000000,1.000000,3.000000,1.000000


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

def transform_X(X):
    X = X[features]
    # convert to numeric, changing literals to NaN
    for f in num_features:
        X[f] = pd.to_numeric(X[f], errors='coerce')
    return X

X_train = transform_X(X_train)
X_dev = transform_X(X_dev)
X_val = transform_X(X_val)
X_test = transform_X(X_test)

# https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('oh_enc', OneHotEncoder(handle_unknown='ignore'))
])
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)
clf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100))
#     ('classifier', LogisticRegression(C=.001))
])

/Users/steed/miniconda3/envs/hmm/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Using just the labels (no label model):

In [20]:
from snorkel.analysis import metric_score

def test_accuracy(clf):
    preds_test_dev = np.round(clf.predict(X_test))
    test_acc = metric_score(golds=y_test, preds=preds_test_dev, metric="accuracy")
    print(f"Test Accuracy: {test_acc * 100:.1f}%")
    
clf.fit(X=X_train, y=y_train)
test_accuracy(clf)

Test Accuracy: 85.5%


Using the label model:

In [21]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=X_train, y=model_label.predict_proba(L=L_train), L=L_train
)
df_train_filtered

,PedPed,CrossingSignal,NumberOfCharacters_int,Man_int,Woman_int,Pregnant_int,Stroller_int,OldMan_int,OldWoman_int,Boy_int,...,LargeMan_noint,Criminal_noint,MaleExecutive_noint,FemaleExecutive_noint,FemaleAthlete_noint,MaleAthlete_noint,FemaleDoctor_noint,MaleDoctor_noint,Dog_noint,Cat_noint
ResponseID,,,,,,,,,,,,,,,,,,,,,
NsfTHxKZRYe37byxk,0,1,4,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,1,0
WBneJstQdv9rbuvt4,0,1,3,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
bk5KMmomyLuQoZMz6,1,2,5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
nP7jf6qekrf89PYPg,0,0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
FX2whvdz7om5xkQTD,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
srAQBkgphCtumFxEC,0,1,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
YvoTQ4zwoAhGpNWTC,0,2,3,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
JEeL28JCm6PSuMFnt,0,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
from snorkel.utils import probs_to_preds

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)
clf.fit(X=df_train_filtered, y=preds_train_filtered)
test_accuracy(clf)

Test Accuracy: 66.9%
